In [1]:
import praw
import pandas as pd
from transformers import RobertaTokenizer
import tensorflow as tf
from transformers import TFRobertaForSequenceClassification
import tensorflow_datasets as tfds

C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
reddit = praw.Reddit(client_id='c61aYydHGTRGkw',
                     client_secret='hEw2j00lh_w2inKcTAlcqq_kJT0',
                     user_agent='android:com.example.myredditapp:v1.2.3')

# get 10 hot posts from the MachineLearning subreddit
top_posts = reddit.subreddit('showerthoughts').top('week', limit=10)


Version 7.0.0 of praw is outdated. Version 7.1.0 was released Tuesday June 23, 2020.


In [3]:

max_length = 100
batch_size = 64

In [4]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [5]:
def convert_example_to_feature(review):
    # combine step for tokenization, WordPiece vector mapping and will
    # add also special tokens and truncate reviews longer than our max length
    return roberta_tokenizer.encode_plus(review,
                                         add_special_tokens=True,
                                         max_length=max_length,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         )


# map to the expected input to TFRobertaForSequenceClassification, see here
def map_example_to_dict(input_ids, attention_masks, label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label


def encode_examples(ds, limit=-1):
    # Prepare Input list
    input_ids_list = []
    attention_mask_list = []
    label_list = []

    if (limit > 0):
        ds = ds.take(limit)

    for review, label in tfds.as_numpy(ds):
        bert_input = convert_example_to_feature(review.decode())
        input_ids_list.append(bert_input['input_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                               attention_mask_list,
                                               label_list)).map(map_example_to_dict)


def replies_of(top_level_comment, comment_list):
    if len(top_level_comment.replies) == 0:
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                comment_list.append(str(comment.body))
            except:
                continue
            replies_of(comment, comment_list)


In [6]:
# load saved model
model = TFRobertaForSequenceClassification.from_pretrained('reddit_model5')

In [19]:
#list_of_subreddit = ['showerthoughts', 'askmen', 'askreddit', 'jokes', 'worldnews']
list_of_subreddit = ['showerthoughts']

In [20]:
for j in list_of_subreddit:
    # get 10 hot posts from the MachineLearning subreddit
    top_posts = reddit.subreddit(j).top('week', limit=10)
    comment_list = []
    # save subreddit comments in dataframe
    for submission in top_posts:
        submission_comm = reddit.submission(id=submission.id)

        for count, top_level_comment in enumerate(submission_comm.comments):
            try:
                replies_of(top_level_comment, comment_list)
            except:
                continue

    comment_dataframe = pd.DataFrame(comment_list, columns=['Comments'])
    comment_dataframe['label'] = 0
  
  
    
    # prepare data as per RoBERTa model input
    submission_sentences_modified = tf.data.Dataset.from_tensor_slices((comment_dataframe['Comments'],
                                                                        comment_dataframe['label']))    
    
    ds_submission_encoded = encode_examples(submission_sentences_modified).batch(batch_size)

    
    # predict sentiment of Reddit comments
    submission_pre = tf.nn.softmax(model.predict(ds_submission_encoded))
    submission_pre_argmax = tf.math.argmax(submission_pre, axis=1)
    comment_dataframe['label'] = submission_pre_argmax

    negative_comments_count = comment_dataframe[comment_dataframe['label'] == 1].count()
    positive_comments_count = comment_dataframe[comment_dataframe['label'] == 0].count()

    print(f"overall sentiment of subreddit r/{j} are Positive comments: {positive_comments_count}"
          f" Negative comments: {negative_comments_count}")
    

overall sentiment of subreddit r/showerthoughts are Positive comments: Comments    2752
label       2752
dtype: int64 Negative comments: Comments    8
label       8
dtype: int64


In [18]:
comment_dataframe.head(50)
comment_dataframe.shape

(3966, 2)

In [22]:
pd.set_option('display.max_colwidth', -1)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [23]:
comment_dataframe[comment_dataframe['label'] == 1].head(100)

,Comments,label
411,How does cultural appropriation value them as gifts?,1
514,No hate but what’s consent,1
1631,"This right here, this is the exact rhetoric that Witchifer was talking about. That rhetoric you are using, is exactly how those Youtubers talking about flat-earth and antivaxx start their videos.",1
1730,"It's also prone to biases, corruption, incompetence and misinterpretation.",1
1770,Pseudoscience isn't science though. The scientific consensus supports the validity of trans peoples identities.\n[https://blogs.scientificamerican.com/voices/stop-using-phony-science-to-justify-transphobia/](),1
1843,r/thalassophobia,1
1862,thalassophobia intensifies...,1
2551,"Grey-asexual is someone who experiences sexual attraction very rarely, as opposed to a gay asexual who may also be known as a homoromantic asexual.",1


In [24]:
comment_dataframe[comment_dataframe['label'] == 0].head(100)

,Comments,label
0,[deleted],0
1,"Right now there is some alien species somewhere who shit gold and trade with poop, wondering the same.",0
2,This could be a science fiction novel. I would read the ~~shit~~poop out of it,0
3,r/WritingPrompts,0
4,There's already a storyline on American dad that revolve around Roger pooping gold and gems,0
...,...,...
95,What a ~~country~~ galaxy!,0
96,"Fantastic username, by the way.",0
97,Or entire solar systems where every planet is colonized with clams resting in sandy banks with moons populated with trees dripping with sap.,0
98,Ok but what about space weed.,0


In [26]:
print(type(negative_comments_count))

<class 'pandas.core.series.Series'>
